In [1]:
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pdb

In [2]:
from easydict import EasyDict as edict
from matplotlib import pyplot as plt
from torchvision import transforms as trans
from PIL import Image
from pathlib import Path
import numpy as np
# np.seterr(all='raise')
import torch
from torch import nn, optim
# import torch.nn.functional as F
from utils.vis_utils import *
from utils.box_utils import *
from utils.dataset_tools import *
from utils.utils import *
from tensorboardX import SummaryWriter
from tqdm import tqdm_notebook as tqdm
from imgaug import augmenters as iaa
from torch.utils.data import DataLoader
from models.Yolo_model import Yolo_model, build_targets, yolo_loss
from Yolo_learner_V2 import Yolo

In [3]:
conf = edict()

conf.coco_anchors = [[10, 13], [16, 30], [33, 23], [30, 61], [62, 45],
                     [59, 119], [116, 90], [156, 198], [373, 326]]
conf.train_path = Path('/home/f/nvme/coco2017/train2017/')
conf.train_anno_path = Path(
    '/home/f/nvme/coco2017/annotations/instances_train2017.json')
conf.val_path = Path('/home/f/nvme/coco2017/val2017/')
conf.val_anno_path = Path(
    '/home/f/nvme/coco2017/annotations/instances_val2017.json')
conf.log_path = Path('/home/f/learning/yolo/log')
conf.model_path = Path('/home/f/learning/yolo/model')
conf.save_path = Path('/home/f/learning/yolo/save')
conf.ids_path = 'data/ids.npy'

conf.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

maps,correct_id_2_class = get_id_maps(conf)

loading annotations into memory...
Done (t=9.48s)
creating index...
index created!


In [4]:
conf.num_anchors = 3
conf.batch_size = 16
conf.resolutions = [224,288,352,416,480,544,608]
conf.res_2_idx = edict({'ft':0, '224':1, '288':2, '352':3, '416':4, '480':5, '544':6, '608':7})
conf.idx_2_res = edict()
for k,v in conf.res_2_idx.items():
    conf.idx_2_res[str(v)] = k
conf.input_size = 416
conf.scales = [32, 16, 8]

conf.running_norm = 0.
# conf.gdclip = 3000.
conf.num_workers = 4
conf.batch_size = 16
conf.gdclip = None
conf.coord_scale = 2.
conf.noobject_scale = 0.5
conf.object_scale = 5
conf.class_scale = 5.
conf.ignore_thresh = 0.5
conf.evaluate_iou_threshold = 0.5
conf.predict_confidence_threshold = 0.5
conf.pred_nms_iou_threshold = 0.5
conf.object_only = True
conf.warm_up_img_num = 12800

model = Yolo_model(conf)
model.to(conf.device)
conf.mean = model.res50_pyramid.model.mean
conf.std = model.res50_pyramid.model.std

conf.mse_loss = nn.MSELoss(size_average=False)
conf.bce_loss = nn.BCEWithLogitsLoss(size_average=False)

# conf.eva_batches = 5
# conf.board_loss_every = 5
# conf.evaluate_every = 5
# conf.board_pred_image_every = 5
# conf.save_every = 5

train_ds = Coco_dataset(conf, conf.train_path, conf.train_anno_path, maps)
train_loader = DataLoader(
    train_ds,
    batch_size=conf.batch_size,
    shuffle=True,
    collate_fn=coco_collate_fn,
    pin_memory=False,
    num_workers=conf.num_workers)
conf.eva_batches = 100
conf.board_loss_every = len(train_loader) // 100
conf.evaluate_every = len(train_loader) // 10
conf.board_pred_image_every = len(train_loader) // 2
conf.save_every = len(train_loader) // 2
conf.board_grad_norm = len(train_loader) // 10
val_dataset = datasets.CocoDetection(conf.val_path, conf.val_anno_path)
val_dataset.maps = maps
conf.transform_test = trans.Compose([
    trans.Resize([conf.input_size, conf.input_size]),
    trans.ToTensor(),
    trans.Normalize(conf.mean, conf.std)
])
val_loader = Coco_loader(
    conf,
    val_dataset,
    conf.transform_test,
    batch_size=conf.batch_size,
    hflip=False,
    shuffle=False)

loading annotations into memory...
Done (t=9.05s)
creating index...
index created!
loading annotations into memory...
Done (t=0.31s)
creating index...
index created!


In [5]:
paras = [*model.parameters()][159:]

optimizer = optim.SGD(paras,lr=1e-4,momentum=0.9,weight_decay=1e-4)

In [6]:
yolo = Yolo(conf,model,train_loader,val_loader,optimizer)

In [7]:
yolo.model.load_state_dict(
    torch.load(
        conf.model_path /
        '2018-07-02-08-26_val_loss:195.18045036315917_model_seen:351483_step:22176_None.pth'
    ))
yolo.optimizer.load_state_dict(
    torch.load(
        conf.model_path /
        '2018-07-02-08-26_val_loss:195.18045036315917_optimizer_seen:351483_step:22176_None.pth'
    ))
yolo.seen = 351483
yolo.steps[0] = 22176

In [8]:
yolo.train(conf,2)

Process Process-2:
Process Process-1:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 57, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 57, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 57, in _worker_loop
    samples = collate_fn([dat

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-bc7ef83bd3bc>", line 1, in <module>
    yolo.train(conf,2)
  File "/root/Notebooks/YOLOv3/Yolo_learner_V2.py", line 283, in train
    self.model.head.anchors_group, warm_up)
  File "/root/Notebooks/YOLOv3/models/Yolo_model.py", line 183, in build_targets
    targets[l][b,best_anchor_idx_inGroup,gt_ij[1],gt_ij[0],1] = true_xy[1]
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 1863, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call la

KeyboardInterrupt: 

In [ ]:
writer_544 = SummaryWriter(conf.log_path/'size_544')
writer_480 = SummaryWriter(conf.log_path/'size_480')
writer_416 = SummaryWriter(conf.log_path/'size_416')
writer_352 = SummaryWriter(conf.log_path/'size_352')
writer_288 = SummaryWriter(conf.log_path/'size_288')

In [ ]:
yolo.update_size(conf,writer_416)